In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
trials_info = pd.read_csv('trials_info2', sep=',', index_col=0)  
trial_timings = pd.read_csv('trial_timings2', sep=',', index_col=0) 
spike_times = pd.read_pickle('./kluster2.kkp') 
trials_info = trial_timings.join(trials_info) 

In [3]:
st=np.unique(trials_info['stim_name'].values).tolist()

In [4]:
#trials_info[trials_info['go_or_nogo']=='go']

In [1]:
#trials_info

In [6]:
#trials_info

In [6]:
def passing(N):
    l=[]
    for item in N:
        x=item*1000/1000
        y="%.3f" % x
        z=float(y)
        l.append(z)
        
        
    return l

In [7]:
gr=spike_times[spike_times['group']>4]
gr=gr.sort_values(by=['time']).reset_index()
u=np.unique(gr['unit'].values)

In [8]:
time_axes=np.zeros(8000000)

In [9]:
def unit(gr,units, time_axes):
    
    
    time=time_axes.astype(int).reshape(-1,1)
    
                      
    for number in units:
        
        val=np.array(passing(gr[gr['unit']==number]['time'].values))*1000
        
        h=val.astype(int).tolist()
        
        time_ax=np.zeros(8000000).astype(int)
        
        time_ax[h]=1
        
        time=np.append(time, time_ax.reshape(-1,1), axis=1)
        
    return time

In [10]:
units_col=unit(gr, u, time_axes)

In [11]:
np.unique(units_col, return_counts=True)

(array([0, 1]), array([415135408,    864592], dtype=int64))

In [12]:
df_units=pd.DataFrame(units_col)
df_units=df_units.drop(df_units.columns[[0]], axis = 1)

In [13]:
df_units.columns=u

In [14]:
def all_stimulus(st, trials_info):
    df=pd.DataFrame(np.zeros(8000000))
    
    for value in st: 
        ar=np.array(passing(np.sort(trials_info[trials_info['stim_name']==value]['stim_onset'].values)))*1000
        ev_ar=evoked(ar)
        df[value]=ev_ar
    return df
        

In [20]:
stim_t=all_stimulus(st, trials_info)

In [15]:
def modify_array(h):
    ar=h.astype(int)
    val=np.unique(ar)
   
    
    for v in val:
       
        ind=np.where(ar==v)[0].astype(int).tolist()
        ins=np.arange(int(v), int(v)+251, 1).astype(int)
        ar=np.insert(ar.astype(int), ind[0], ins)
        
    return ar
        

In [16]:
def evoked(array):
    

    t=np.zeros(8000000).astype(int)
    indices=modify_array(array)
    ind=indices.astype(int).tolist()
    t[ind]=1
    
    return t
    

In [17]:
def pres(start, d):
    ar=start.astype(int)
    val=np.unique(ar)
    counter=0
    for v in val:
        
        ind=np.where(ar==v)[0].astype(int).tolist()
        ins=np.arange(int(v), int(v)+int(d[counter]), 1).astype(int)
        ar=np.insert(ar.astype(int), ind[0], ins)
        counter=counter+1
    return ar
        

In [18]:
def pres_evoked(array, d):
    
    

    t=np.zeros(8000000).astype(int)
    indices=pres(array, d)
    ind=indices.astype(int).tolist()
    t[ind]=1
    
    return t
    

In [19]:
def all_prestimulus(st, trials_info):
    df=pd.DataFrame(np.zeros(8000000))
    for value in st:
        pres=trials_info[trials_info['stim_name']==value]['cpoke_start'].values
        d=(np.array(passing(trials_info[trials_info['stim_name']==value]['stim_onset'].values))-np.array(passing(trials_info[trials_info['stim_name']==value]['cpoke_start'].values)))*1000
        d=d.astype(int)
        pres=np.array(passing(pres))*1000
        prestimulus=pres_evoked(pres, d)
        df[value+"pres"]=prestimulus
    return df
        

In [21]:
pres_t=all_prestimulus(st, trials_info)

In [22]:
df=pd.concat([df_units, pres_t.drop(pres_t.columns[[0]], axis = 1), stim_t.drop(stim_t.columns[[0]], axis = 1)], axis=1)

In [23]:
df.columns

Index([           501,            502,            503,            504,
                  505,            506,            507,            508,
                  509,            510,            511,            512,
                  601,            602,            604,            605,
                  606,            607,            608,            609,
                  611,            612,            701,            702,
                  706,            707,            708,            710,
                  711,            712,            713,            715,
                  801,            802,            803,            804,
                  805,            806,            807,            808,
                  809,            810,            811,            812,
                  813,            814,            815,            816,
                  817,            818,            819, 'hi_pc_nopres',
       'le_hi_lcpres', 'le_hi_pcpres', 'le_lc_gopres', 'le_lo_lcpres',
      

In [24]:
df['indexes']=df.index

In [ ]:
for i in range (0, 50):
    
    
    stim=df[df['le_hi_lc']==1]
    single_stim=stim.iloc[:, i].values.reshape(87,251)
    name=stim.iloc[:,i].head().name
    ind=stim.iloc[:,i].index.values.reshape(87,251)
    idd=ind[:,0]
    mean_count=per_trial(df, idd, single_stim).tolist()
    m_r=[l/86 for l in mean_count]
    pd.DataFrame(m_r).to_csv(str(name)+'le_hi_lc.csv')
        

In [44]:

   
  
  
    
for i in range (3, 50):
    for n in range(51, 75):
        if not str(df.iloc[:,n].head().name).endswith('pres'):
            number=df.iloc[:,n].head().name
            
            stim=df[df[str(df.iloc[:,n].head().name)]==1]
            ln=len(stim)/251
            single_stim=stim.iloc[:, i].values.reshape(int(ln), 251)
            name=stim.iloc[:,i].head().name
            ind=stim.iloc[:,i].index.values.reshape(int(ln), 251)
            idd=ind[:,0]
            mean_count=per_trial(df, idd, single_stim, number, ln).tolist()
            m_r=[l/(int(ln)-1) for l in mean_count]
            pd.DataFrame(m_r).to_csv(str(name)+str(number)+'.csv')
        
                 
                                 
            
       
                             
                               
                                
            
            
            
        
        
        
        

In [42]:
 def per_trial(df, idd, single_stim, number, ln):
        b=np.zeros(13)
      
        
        for t in range(1, int(ln)):
            
            
            
            
            temp_t=single_stim[t].tolist()
            pres_t=df[(df['indexes'] < int(idd[t])) & (df[str(number)+'pres']==1)&(df['indexes'] > int(idd[t-1]))]
            pres=pres_t.iloc[:, i].values.tolist()
            all=pres+temp_t
            all_equal=all+ np.zeros(650-len(all)).tolist()
                                 
            a=np.array(all_equal).reshape(13, 50).tolist()
            a=[sub.count(1)*20 for sub in a ]
            b=b+np.array(a)    
            
        return b 
                                 
                       